****Assignment Week 4 - Cafes and Parks in Toronto****

Importing the librairies

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


Using geopy library to get the latitude and longitude values of Toronto

In [37]:
city = 'Vancouver, Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


Define Foursquare Credentials and Version

In [38]:
CLIENT_ID = 'MCNCWJCVWTFVMY1NRUQ2JA4ZI3YOA12LPD1LY1FFQFLZP2HV' # your Foursquare ID
CLIENT_SECRET = 'WRV4A3FY5ALJGIEBQPDAJ2EWMYU3HPJ3I3AQ3FN0KC3J50PQ' # your Foursquare Secret
ACCESS_TOKEN = '5EVWMVOKVQHFAERECXJTJFDLBFYN1BGFERLLJ0DQESPKXDUC' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MCNCWJCVWTFVMY1NRUQ2JA4ZI3YOA12LPD1LY1FFQFLZP2HV
CLIENT_SECRET:WRV4A3FY5ALJGIEBQPDAJ2EWMYU3HPJ3I3AQ3FN0KC3J50PQ


Getting Parks and Cafes data

In [39]:
search_query = 'Park'
radius = 10000

url_park = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url_park

'https://api.foursquare.com/v2/venues/search?client_id=MCNCWJCVWTFVMY1NRUQ2JA4ZI3YOA12LPD1LY1FFQFLZP2HV&client_secret=WRV4A3FY5ALJGIEBQPDAJ2EWMYU3HPJ3I3AQ3FN0KC3J50PQ&ll=49.2608724,-123.1139529&oauth_token=5EVWMVOKVQHFAERECXJTJFDLBFYN1BGFERLLJ0DQESPKXDUC&v=20180604&query=Park&radius=10000&limit=30'

In [40]:
park_results = requests.get(url_park).json()

In [41]:
# assign relevant part of JSON to venues
venues = park_results['response']['venues']

# tranform venues into a dataframe
df_park = json_normalize(venues)
df_park.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4be73b80910020a16435d514,Jonathon Rogers Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1614194097,False,110 E 7th Ave,49.264080,-123.108029,"[{'label': 'display', 'lat': 49.26407997044988...",559,V6G 1Z4,CA,Vancouver,BC,Canada,"[110 E 7th Ave, Vancouver BC V6G 1Z4]",NaN,NaN,NaN
1,4ed90eae02d5244e2ed2252d,Pay Park,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",v-1614194097,False,Willow St,49.262632,-123.121882,"[{'label': 'display', 'lat': 49.26263160075835...",608,NaN,CA,Vancouver,BC,Canada,"[Willow St (at W Broadway), Vancouver BC]",at W Broadway,NaN,NaN
2,4aada430f964a5201f6120e3,Charleson Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1614194097,False,999 Charleson St,49.266880,-123.124681,"[{'label': 'display', 'lat': 49.26688043290664...",1026,V5Z,CA,Vancouver,BC,Canada,"[999 Charleson St, Vancouver BC V5Z]",NaN,NaN,NaN
3,4aa9d826f964a5206b5520e3,Cooper's Park,"[{'id': '4bf58dd8d48988d1e5941735', 'name': 'D...",v-1614194097,False,1020 Marinaside Crescent,49.273547,-123.113751,"[{'label': 'display', 'lat': 49.27354702489317...",1411,NaN,CA,Vancouver,BC,Canada,"[1020 Marinaside Crescent (Cambie Bridge), Van...",Cambie Bridge,NaN,NaN
4,4aec5fd7f964a52075c621e3,Emery Barnes Park & Playground,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1614194097,False,1199 Richards St,49.275373,-123.123400,"[{'label': 'display', 'lat': 49.2753733570921,...",1754,V6G 1Z4,CA,Vancouver,BC,Canada,"[1199 Richards St (at Davie St), Vancouver BC ...",at Davie St,NaN,NaN


In [43]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_park.columns if col.startswith('location.')] + ['id']
dataframe_filtered = df_park.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

park_final= dataframe_filtered.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress','labeledLatLngs','neighborhood', 'id'], axis=1)
array= ['Park','Garden','Other Great Outdoors','Playground']
park_final= park_final.loc[park_final['categories'].isin(array)]
park_final

,name,categories,address,lat,lng,postalCode,state
0,Jonathon Rogers Park,Park,110 E 7th Ave,49.264080,-123.108029,V6G 1Z4,BC
2,Charleson Park,Park,999 Charleson St,49.266880,-123.124681,V5Z,BC
4,Emery Barnes Park & Playground,Park,1199 Richards St,49.275373,-123.123400,V6G 1Z4,BC
5,David Lam Park,Park,1300 Pacific Blvd.,49.272467,-123.123866,V6Z 2Y1,BC
8,Douglas Park & Playground,Park,W 22nd Ave,49.253132,-123.122123,NaN,BC
10,Nelson Park,Park,1030 Bute St.,49.282560,-123.129230,V6E 1Y7,BC
11,George Wainborn Park,Park,500 Beach Cres.,49.272383,-123.129433,NaN,BC
12,Delamont Park,Park,2091 W 7th Ave,49.265822,-123.152312,NaN,BC
13,Queen Elizabeth Park,Park,4600 Cambie St,49.241565,-123.113355,NaN,BC
14,Prince Edward Park,Park,3773 Prince Edward Street,49.251588,-123.097558,NaN,BC


In [44]:
search_query = 'Cafe'
radius = 10000

url_cafe = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url_cafe

'https://api.foursquare.com/v2/venues/search?client_id=MCNCWJCVWTFVMY1NRUQ2JA4ZI3YOA12LPD1LY1FFQFLZP2HV&client_secret=WRV4A3FY5ALJGIEBQPDAJ2EWMYU3HPJ3I3AQ3FN0KC3J50PQ&ll=49.2608724,-123.1139529&oauth_token=5EVWMVOKVQHFAERECXJTJFDLBFYN1BGFERLLJ0DQESPKXDUC&v=20180604&query=Cafe&radius=10000&limit=30'

In [45]:
cafe_results = requests.get(url_cafe).json()

In [46]:
# assign relevant part of JSON to venues
venues_cafe = cafe_results['response']['venues']

# tranform venues into a dataframe
df_cafe = json_normalize(venues_cafe)
df_cafe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.neighborhood,venuePage.id
0,51dc68ad498e8db03f7d7ee5,Cafè Molto Bella,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1614194134,False,3010 Cambie St.,14th Ave.,49.258539,-123.115098,"[{'label': 'display', 'lat': 49.258539, 'lng':...",272,CA,Vancouver,BC,Canada,"[3010 Cambie St. (14th Ave.), Vancouver BC]",NaN,NaN,NaN
1,4adbdddff964a520ec2a21e3,Café Gloucester 告羅士打餐廳,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1614194134,False,3338 Cambie St.,at W 17th Ave.,49.255528,-123.115040,"[{'label': 'display', 'lat': 49.25552810968816...",600,CA,Vancouver,BC,Canada,"[3338 Cambie St. (at W 17th Ave.), Vancouver B...",V5Z 2W5,NaN,NaN
2,4e8cba56d3e3720da3aaa38b,Golden Pair Fusion Cafe 皇牌魚湯米線,"[{'id': '4bf58dd8d48988d1d1941735', 'name': 'N...",v-1614194134,False,546 West Broadway,Cambie Street,49.263098,-123.116492,"[{'label': 'display', 'lat': 49.26309795483614...",308,CA,Vancouver,BC,Canada,"[546 West Broadway (Cambie Street), Vancouver BC]",NaN,NaN,NaN
3,4ac9234af964a520d2be20e3,Soba Cafe,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1614194134,False,2533 Heather St,West Broadway,49.263147,-123.119976,"[{'label': 'display', 'lat': 49.26314725085579...",505,CA,Vancouver,BC,Canada,"[2533 Heather St (West Broadway), Vancouver BC...",V5Z,NaN,NaN
4,4b2d26b8f964a5208ecf24e3,Dutch Wooden Shoe Cafe,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1614194134,False,3292 Cambie St.,at W 17 Ave.,49.256119,-123.115090,"[{'label': 'display', 'lat': 49.25611944213464...",535,CA,Vancouver,BC,Canada,"[3292 Cambie St. (at W 17 Ave.), Vancouver BC ...",V5Z 2W4,NaN,NaN


In [53]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns2 = ['name', 'categories'] + [col for col in df_cafe.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = df_cafe.loc[:, filtered_columns2]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list2 = row['categories']
    except:
        categories_list2 = row['venue.categories']
        
    if len(categories_list2) == 0:
        return None
    else:
        return categories_list2[0]['name']

# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

cafe_final= dataframe_filtered2.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress','labeledLatLngs','neighborhood', 'id'], axis=1)
array= ['Café','Coffee Shop']
cafe_final= cafe_final.loc[cafe_final['categories'].isin(array)]
cafe_final

,name,categories,address,lat,lng,state,postalCode
0,Cafè Molto Bella,Café,3010 Cambie St.,49.258539,-123.115098,BC,NaN
3,Soba Cafe,Café,2533 Heather St,49.263147,-123.119976,BC,V5Z
5,Tandem Bike Cafe,Café,3195 Heather St,49.257090,-123.120223,BC,NaN
7,Sweetery Cafe + Dessert,Café,340 West 2nd Avenue,49.268215,-123.111666,BC,V5Y 1C8
9,Canuel catering cafe,Café,Vancouver City Hall Sub Basement,49.260815,-123.113459,BC,NaN
10,Wicked Cafe,Coffee Shop,1399 W 7th Ave,49.265346,-123.135991,BC,V6H 1B8
12,PRADO Cafe,Coffee Shop,100 W. Hastings St.,49.281908,-123.107295,BC,NaN
15,Kranky Cafe,Café,228 E 4th Ave,49.267333,-123.099977,BC,V5T 1G5
18,Giovane Café + Market + Eatery,Café,1038 Canada Pl,49.287806,-123.117070,BC,V6C 0B9
26,Cafe Butterfly Garden,Café,575 W Georgia St,49.281303,-123.117195,BC,V6B 1Z5


Map to visualize Parks and Cafe clustered together

In [48]:
final_df = pd.concat([park_final, cafe_final], ignore_index=True)
final_df

,name,categories,address,lat,lng,postalCode,state
0,Jonathon Rogers Park,Park,110 E 7th Ave,49.264080,-123.108029,V6G 1Z4,BC
1,Charleson Park,Park,999 Charleson St,49.266880,-123.124681,V5Z,BC
2,Emery Barnes Park & Playground,Park,1199 Richards St,49.275373,-123.123400,V6G 1Z4,BC
3,David Lam Park,Park,1300 Pacific Blvd.,49.272467,-123.123866,V6Z 2Y1,BC
4,Douglas Park & Playground,Park,W 22nd Ave,49.253132,-123.122123,NaN,BC
5,Nelson Park,Park,1030 Bute St.,49.282560,-123.129230,V6E 1Y7,BC
6,George Wainborn Park,Park,500 Beach Cres.,49.272383,-123.129433,NaN,BC
7,Delamont Park,Park,2091 W 7th Ave,49.265822,-123.152312,NaN,BC
8,Queen Elizabeth Park,Park,4600 Cambie St,49.241565,-123.113355,NaN,BC
9,Prince Edward Park,Park,3773 Prince Edward Street,49.251588,-123.097558,NaN,BC


In [50]:
final_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(final_df['lat'], final_df['lng'], final_df['name'], final_df['categories'], final_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(final_map)  
    
final_map